# Imports

In [1]:
from tqdm import tqdm
import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import time, sys

# Data Files

In [2]:
RF_Data = pd.read_csv("C:\\Users\\Ritvik Bale\\Documents\\Systemic Risk Research\\Risk Factor Data v1.csv")
Bank_Data = pd.read_csv("C:\\Users\\Ritvik Bale\\Documents\\Systemic Risk Research\\Bank data v1 2020-08-24.csv")

RF_Data = pd.DataFrame(RF_Data)
Bank_Data = pd.DataFrame(Bank_Data)

# Risk Factor Data Formatting

In [3]:
Return_Table = []
Return_Table = pd.DataFrame(Return_Table)

dates = []

BOND = []
CREDIT = []
SP500 = []
CMDTY = []
DVIX = []
DHOUSE = []

In [4]:
for i in RF_Data["Date"]:
    
    index = RF_Data[RF_Data["Date"] == i].index[0]
    month = (datetime.datetime.strptime(i, "%Y-%m-%d")).month
    
    if month == 2:
        dates.append(i)
        

In [5]:
year = []
month = []
day = []

for i in dates:
    
    x = datetime.datetime.strptime(str(i), "%Y-%m-%d")
    
    year.append(x.year)
    month.append(x.month)
    day.append(x.day)

In [6]:
for i in dates:
    
    date = RF_Data[RF_Data["Date"] == i].index[0]
    
    BOND.append(RF_Data.loc[date, "BOND"])
    CREDIT.append(RF_Data.loc[date, "CREDIT"])
    SP500.append(RF_Data.loc[date, "SP500"])
    CMDTY.append(RF_Data.loc[date, "CMDTY"])
    DVIX.append(RF_Data.loc[date, "DVIX"])
    DHOUSE.append(RF_Data.loc[date, "DHOUSE"])

In [7]:
Return_Table["BOND"] = BOND
Return_Table["CREDIT"] = CREDIT
Return_Table["SP500"] = SP500
Return_Table["CMDTY"] = CMDTY
Return_Table["DVIX"] = DVIX
Return_Table["DHOUSE"] = DHOUSE

Return_Table["Year"] = year
Return_Table["Month"] = month
Return_Table["Day"] = day

In [8]:
Return_Table.index = dates

In [10]:
Return_Table.head()

,BOND,CREDIT,SP500,CMDTY,DVIX,DHOUSE,Year,Month,Day
1975-02-01,0.0388,-14.02,0.1386,-0.1654,-3.8780,0.0024,1975,2,1
1976-02-01,0.0053,-8.10,0.1212,-0.0268,0.2137,0.0026,1976,2,1
1977-02-01,-0.0235,-5.38,-0.0385,0.0090,0.9400,0.0114,1977,2,1
1978-02-01,-0.0089,-4.60,-0.0502,0.0383,0.1433,0.0117,1978,2,1
1979-02-01,0.0136,-7.32,0.0443,0.0619,-4.1077,0.0132,1979,2,1


# Bank Data Formatting

In [11]:
date = []
year = []
month = []
day = []

for i in Bank_Data["date"]:
    x = datetime.datetime.strptime(str(i), "%Y%m%d")
    
    date.append(x)
    year.append(x.year)
    month.append(x.month)
    day.append(x.day)

In [12]:
Bank_Data["date"] = date
Bank_Data["Year"] = year
Bank_Data["Month"] = month
Bank_Data["Day"] = day

In [13]:
Bank_Returns = []

permno = Bank_Data["PERMNO"] == 10071
month = Bank_Data["Month"] == 2

Single_Firm = Bank_Data[permno & month]

In [14]:
Bank_Returns = Single_Firm["RET"]

Bank_Returns = pd.DataFrame(Bank_Returns)

In [15]:
Year = Single_Firm["Year"]
Month = Single_Firm["Month"]
Day = Single_Firm["Day"]

In [16]:
Bank_Returns["Year"] = Year
Bank_Returns["Month"] = Month
Bank_Returns["Day"] = Day

In [17]:
Bank_Returns.index = Single_Firm["date"]

In [18]:
bond = []
credit = []
sp500 = []
cmdty = []
dvix = []
dhouse = []

for index, i in Bank_Returns.iterrows():
    year = Return_Table["Year"] == i['Year']
    month = Return_Table["Month"] == i['Month']
    
    bond.append((Return_Table["BOND"][year & month]))
    credit.append((Return_Table["CREDIT"][year & month]))
    sp500.append((Return_Table["SP500"][year & month]))
    cmdty.append((Return_Table["CMDTY"][year & month]))
    dvix.append((Return_Table["DVIX"][year & month]))
    dhouse.append((Return_Table["DHOUSE"][year & month]))

In [19]:
bond = np.array(bond)
credit = np.array(credit)
sp500 = np.array(sp500)
cmdty = np.array(cmdty)
dvix = np.array(dvix)
dhouse = np.array(dhouse)

In [20]:
Bank_Returns["BOND"] = bond
Bank_Returns["CREDIT"] = credit
Bank_Returns["SP500"] = sp500
Bank_Returns["CMDTY"] = cmdty
Bank_Returns["DVIX"] = dvix
Bank_Returns["DHOUSE"] = dhouse

In [21]:
Bank_Returns

,RET,Year,Month,Day,BOND,CREDIT,SP500,CMDTY,DVIX,DHOUSE
date,,,,,,,,,,
1993-02-26,-0.026882,1993,2,26,0.0140,-4.991,0.0171,0.0126,0.74,0.0007
1994-02-28,0.005357,1994,2,28,0.0101,-3.097,0.0320,0.0960,4.24,0.0017
1995-02-28,0.093525,1995,2,28,0.0135,-2.870,0.0260,-0.0239,-0.21,0.0013
1996-02-29,0.024615,1996,2,29,0.0037,-3.740,0.0382,-0.0110,4.51,0.0015
1997-02-28,0.096698,1997,2,28,-0.0024,-2.643,0.0633,-0.0195,1.63,0.0027


In [22]:
y = Bank_Returns['RET']
x = Bank_Returns[['SP500']]

In [23]:
linear_regression = LinearRegression()

In [24]:
linear_regression.fit(x,y)

LinearRegression()

In [25]:
y_pred = linear_regression.predict(x)
y_pred

array([0.00319555, 0.03219989, 0.02052029, 0.04426881, 0.09312847])

In [26]:
print(linear_regression.coef_)

[1.94659995]
